### Book Library Scraping demo website

In [13]:
import requests
from bs4 import BeautifulSoup

In [14]:
class BookLibrary:
    def __init__(self):
        self.books = []

    def addBook(self, title, price, rate):
        self.books.append({'title': title, 'price': price, 'rate': rate})

    def getBooksOrdered(self, order_by):
        if order_by == 'rate':
            return sorted(self.books, key=lambda x: x['rate'], reverse=True)
        elif order_by == 'alphabet':
            return sorted(self.books, key=lambda x: x['title'])
        elif order_by == 'price_desc':
            return sorted(self.books, key=lambda x: x['price'], reverse=True)
        elif order_by == 'price_asc':
            return sorted(self.books, key=lambda x: x['price'])
        else:
            print("Invalid order_by parameter.")
            return None

In [15]:
library = BookLibrary()

In [16]:
pageNumber = 1
rateList = ['None','One','Two','Three','Four','Five']


while True:
    url = f'https://books.toscrape.com/catalogue/page-{pageNumber}.html'
    
    response = requests.get(url)
    if response.status_code != 200:
        break
    
    html_content = response.content
    
    soup = BeautifulSoup(html_content, 'lxml')
    
    books = soup.find_all('article', class_='product_pod')
   
    for book in books:
        title = book.h3.a['title']
        price = book.find('p', class_='price_color').text[1:]
        rating_tag = book.find('p', class_='star-rating')
        rateWord = rating_tag['class'][-1] if rating_tag else 'None'
        rate = rateList.index(rateWord)
        library.addBook( title, float(price), rate)
    
    pageNumber += 1


In [17]:
import csv

In [18]:
def create_file(path,listOfContacts):
        # Check input type
    if not isinstance(listOfContacts, list):
        raise TypeError('listOfContacts can only be called on a list.')
    
    with open(path, 'w', newline='') as file:
        fieldnames = ['title', 'price', 'rate']
        writer = csv.DictWriter(file, fieldnames=fieldnames)

        writer.writeheader()
        for row in listOfContacts:
            writer.writerow(row)

In [19]:
booksByRate = library.getBooksOrdered('rate')
create_file(f'books_By_Rate_{len(booksByRate)}_Books.csv',booksByRate)

booksByTitleAlphabet = library.getBooksOrdered('alphabet')
create_file(f'books_By_Title_Alphabet_{len(booksByTitleAlphabet)}_Books.csv',booksByTitleAlphabet)

booksByPriceDesc = library.getBooksOrdered('price_desc')
create_file(f'books_By_price_desc_{len(booksByPriceDesc)}_Books.csv',booksByPriceDesc)

booksByPriceAsc = library.getBooksOrdered('price_asc')
create_file(f'books_By_price_asc_{len(booksByPriceAsc)}_Books.csv',booksByPriceAsc)
